In [88]:
from pathlib import Path
import numpy as np
import os
import yaml

# lidarlist_path = Path("/root/map-volume/output/sequences")
lidarlist_path = Path("/root/map-volume/255_19/test_results/sequences")
# lidarlist_path = Path("/root/datasets/kitti_lidarseg/sequences") 20351

lidarlist_path

PosixPath('/root/map-volume/255_19/test_results/sequences')

In [80]:
file_list = os.listdir(lidarlist_path)
file_list

['16', '13', '11', '12', '19', '17', '18', '20', '21', '14', '15']

In [70]:
with open("/root/iSphereFormer/util/semantic-kitti-19.yaml", 'r') as stream:
    semkittiyaml = yaml.safe_load(stream)
splits = semkittiyaml['split']['test']
splits, [str(i) for i in splits]

([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
 ['11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21'])

In [84]:
file_list = os.listdir(lidarlist_path)
num_files = 0
for f in file_list:
    # if f not in [str(i) for i in splits]:
    #     continue
    # file_path = lidarlist_path / f / "velodyne"
    file_path = lidarlist_path / f / "predictions"
    label_list = os.listdir(file_path)
    num_files += len(label_list)

num_files

20351

In [49]:
with open("/root/iSphereFormer/util/semantic-kitti-all.yaml", 'r') as stream:
    semkittiyaml = yaml.safe_load(stream)
learning_map_inv = semkittiyaml['learning_map_inv']
learning_map_inv

{0: 0,
 1: 10,
 2: 11,
 3: 15,
 4: 18,
 5: 20,
 6: 30,
 7: 31,
 8: 32,
 9: 40,
 10: 44,
 11: 48,
 12: 49,
 13: 50,
 14: 51,
 15: 70,
 16: 71,
 17: 72,
 18: 80,
 19: 81,
 20: 252,
 21: 253,
 22: 254,
 23: 255,
 24: 259,
 25: 258}

In [62]:
save_root_path = Path("/root/map-volume/output/sequences")
if not save_root_path.exists():
    save_root_path.mkdir(parents=True, exist_ok=True)
convert_F = lambda label: (0xFFFF << 16) | label
for f in file_list:
    file_path = lidarlist_path / f / "predictions"
    label_list = os.listdir(file_path)
    for label_name in label_list:
        label_path = file_path / label_name
        save_path = save_root_path / label_path.relative_to(lidarlist_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)

        label = np.fromfile(label_path, dtype=np.uint32)
        # print(label)
        annotated_data = np.vectorize(learning_map_inv.__getitem__, otypes=[np.uint32])(label)
        # print(convert_F(label))
        # annotated_data = convert_F(label)
        # print(save_path, save_path.parent)
        # print(np.unique(annotated_data))
        annotated_data.tofile(str(save_path))
        # convert_data = np.fromfile(save_path, dtype=np.uint32)
        # print(convert_data.shape)
        # break
    # break

In [85]:
fpath = lidarlist_path / "16/predictions/000011.label"
f = np.fromfile(fpath, dtype=np.uint32)
# convert_f = lambda label: (0xFFFF << 16) | label
# bf = convert_f(f)
cf = f & 0xFFFF
f.shape, np.unique(cf)

((126855,),
 array([10, 15, 40, 44, 48, 50, 51, 70, 71, 72, 80, 81], dtype=uint32))

In [6]:
def read_ascii_pcd(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    points = []
    header = True
    for line in lines:
        if header:
            if line.startswith("DATA"):
                header = False
        else:
            points.append([float(value) for value in line.strip().split()])

    return np.array(points)

In [32]:
lidarlist_path = Path("/root/datasets/kitti_lidarseg")
file_list = os.listdir(lidarlist_path)
with open("/root/iSphereFormer/util/semantic-kitti-19.yaml", 'r') as stream:
# with open("/root/iSphereFormer/util/semantic-kitti-all.yaml", 'r') as stream:
    semkittiyaml = yaml.safe_load(stream)
splits = semkittiyaml['split']['valid']
learning_map = semkittiyaml['learning_map']
files = []
import glob
for i_folder in splits:
    files += sorted(glob.glob(os.path.join(lidarlist_path, "sequences", str(i_folder).zfill(2), 'labels', "*.label")))
len(files)

4071

In [33]:
from collections import Counter
counts = Counter()

for file_path in files:
    raw_data = np.fromfile(file_path, dtype=np.uint32)
    annotated_data = raw_data & 0xFFFF  # delete high 16 digits binary
    annotated_data = np.vectorize(learning_map.__getitem__)(annotated_data)
    # print(np.unique(annotated_data, return_counts=True))
    # print(annotated_data.shape)
    counts.update(annotated_data)
    # break

# for label, count in counts.items():
    # print(f"label {label}: {count}")
labels = [label for label, _ in counts.items()]
np.sort(np.array(labels).reshape(-1))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [38]:
sorted_items = sorted(counts.items())
sorted_items

[(0, 22321839),
 (1, 30856105),
 (2, 247796),
 (3, 345868),
 (4, 508704),
 (5, 2222268),
 (6, 476946),
 (7, 306860),
 (8, 22737),
 (9, 87963626),
 (10, 5947692),
 (11, 60280496),
 (12, 457650),
 (13, 56878900),
 (14, 12642657),
 (15, 145937016),
 (16, 5503279),
 (17, 64111254),
 (18, 1666427),
 (19, 381442)]

In [35]:
{key: 1 / value if value != 0 else float('inf') for key, value in sorted_items}

{0: 4.4799176268586114e-08,
 1: 3.2408497443212614e-08,
 2: 4.035577652585191e-06,
 3: 2.8912764407230504e-06,
 4: 1.9657797068629302e-06,
 5: 4.4999073019095804e-07,
 6: 2.0966734179550726e-06,
 7: 3.258815094831519e-06,
 8: 4.3981176056647754e-05,
 9: 1.1368335361709623e-08,
 10: 1.681324453250101e-07,
 11: 1.658911366621801e-08,
 12: 2.185075931388616e-06,
 13: 1.7581212013593792e-08,
 14: 7.90972973481761e-08,
 15: 6.852271119480749e-09,
 16: 1.817098497096004e-07,
 17: 1.5597885513204906e-08,
 18: 6.000862924088484e-07,
 19: 2.6216305493364654e-06}

In [40]:
a = np.array([value for key, value in sorted_items if key not in [0]], dtype=np.uint32).reshape(-1)
weights = 1.0 / a
weights = weights / weights.sum()
a.shape, a, weights

((19,),
 array([ 30856105,    247796,    345868,    508704,   2222268,    476946,
           306860,     22737,  87963626,   5947692,  60280496,    457650,
         56878900,  12642657, 145937016,   5503279,  64111254,   1666427,
           381442], dtype=uint32),
 array([5.01559814e-04, 6.24553354e-02, 4.47459213e-02, 3.04227651e-02,
        6.96413857e-03, 3.24485000e-02, 5.04340165e-02, 6.80660698e-01,
        1.75938430e-04, 2.60204837e-03, 2.56736147e-04, 3.38166334e-02,
        2.72090042e-04, 1.22412419e-03, 1.06046997e-04, 2.81217476e-03,
        2.41395720e-04, 9.28704485e-03, 4.05728323e-02]))

In [41]:
freqs = a / a.sum()
median_freq = np.median(freqs[freqs > 0])
weights = median_freq / (freqs + 1e-6)
weights, np.round(weights, 3), weights.shape

(array([7.20192540e-02, 8.95091359e+00, 6.41634658e+00, 4.36439902e+00,
        9.99785509e-01, 4.65471736e+00, 7.23072637e+00, 9.57306450e+01,
        2.52633510e-02, 3.73605405e-01, 3.68651649e-02, 4.85077102e+00,
        3.90698374e-02, 1.75768764e-01, 1.52275331e-02, 4.03773005e-01,
        3.46624241e-02, 1.33317115e+00, 5.81869299e+00]),
 array([7.2000e-02, 8.9510e+00, 6.4160e+00, 4.3640e+00, 1.0000e+00,
        4.6550e+00, 7.2310e+00, 9.5731e+01, 2.5000e-02, 3.7400e-01,
        3.7000e-02, 4.8510e+00, 3.9000e-02, 1.7600e-01, 1.5000e-02,
        4.0400e-01, 3.5000e-02, 1.3330e+00, 5.8190e+00]),
 (19,))

In [12]:
for label, count in counts.items():
    print(f"label {label}: {count}")

label 0: 73587231
label 16: 14189414
label 19: 1441988
label 13: 311802516
label 18: 6712285
label 15: 627195745
label 1: 95302518
label 23: 87766
label 11: 338183720
label 10: 34577789
label 9: 467085589
label 17: 183603141
label 14: 170001681
label 22: 376574
label 5: 5086370
label 20: 4128968
label 8: 13
label 21: 298599
label 2: 391766
label 12: 9173976
label 3: 936031
label 6: 440239
label 7: 5
label 4: 4347360
label 24: 369518
label 25: 238730


In [42]:
from collections import Counter
counts = Counter()

for file_path in files:
    raw_data = np.fromfile(file_path, dtype=np.uint32)
    annotated_data = raw_data & 0xFFFF  # delete high 16 digits binary
    annotated_data = np.vectorize(learning_map.__getitem__)(annotated_data)
    # print(np.unique(annotated_data, return_counts=True))
    # print(annotated_data.shape)
    counts.update(annotated_data)
    # break

for label, count in counts.items():
    print(f"label {label}: {count}")

label 0: 44601775
label 1: 2299457602
label 2: 5500155


In [ ]:
class_counts = np.zeros(3, dtype=np.uint32)
for label, count in counts.items():
    class_counts[label] = count
class_weights = 1.0 / np.log(class_counts + 1)
class_weights = class_weights / np.sum(class_weights)  # 归一化
class_weights

array([0.33875888, 0.27679872, 0.3844424 ])

: 

In [23]:
import random

def generate_random_triplet(a: float, delta: float = 0.1):
    offsets = [random.uniform(-delta, delta) for _ in range(2)]  # 生成两个随机偏移量
    x1 = a + offsets[0]
    x2 = a + offsets[1]
    x3 = 3 * a - (x1 + x2)  # 确保均值为 a
    return x1, x2, x3

In [32]:
generate_random_triplet(83.4, delta=0.5)

(83.68811353318029, 83.70599325288988, 82.80589321392986)

In [89]:
import torch
import time

device = "cuda"
x = torch.randn(1024, 1024, device=device)
y = torch.randn(1024, 1024, device=device)

# FP32 测试
start = time.time()
for _ in range(1000):
    _ = torch.matmul(x, y)
print(f"FP32 time: {time.time() - start:.4f}s")

# FP16 测试（AMP）
with torch.cuda.amp.autocast():
    start = time.time()
    for _ in range(1000):
        _ = torch.matmul(x, y)
    print(f"AMP (FP16) time: {time.time() - start:.4f}s")


/root/.miniconda/envs/mamba-is/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FP32 time: 1.2053s
AMP (FP16) time: 0.1172s
